In [78]:
#FINAL FINAL FINAL

import cvxpy as cp
import numpy as np

num_students = 100
num_universities = 20

np.random.seed(42)  # For reproducibility
student_data = np.column_stack((
    np.round(np.random.uniform(2.0, 4.0, num_students), 2),  
    np.random.randint(1000, 1601, num_students),            
    np.random.randint(1, 101, num_students)               
))

university_requirements = np.column_stack((
    np.round(np.random.uniform(2.5, 4.0, num_universities), 2),  
    np.random.randint(1000, 1601, num_universities),             
    np.random.randint(50, 201, num_universities)                
))

preference_scores = np.random.randint(1, 4, (num_students, num_universities))


# Display the generated data
# print("Student Data (GPA, SAT, Rank):")
# print(student_data)

# print("\nUniversity Requirements (Min GPA, Min SAT, Max Rank):")
# print(university_requirements)

# print("\nPreference Scores:")
# print(preference_scores)

# print("\nUniversity Capacities:")
# print(university_capacities)

x = cp.Variable((num_students, num_universities), boolean=True)

# Sort students
sorted_indices = np.lexsort((student_data[:, 2], student_data[:, 1], -student_data[:, 0]))
sorted_student_data = student_data[sorted_indices]
sorted_preference_scores = preference_scores[sorted_indices]

objective = cp.Maximize(cp.sum(cp.multiply(sorted_preference_scores, x)))

constraints = []
#Eligibility constraint
eligibility_matrix = np.zeros((num_students, num_universities), dtype=int)
for i in range(num_students):
    for j in range(num_universities):
        gpa, sat, rank = sorted_student_data[i]
        min_gpa, min_sat, max_rank = university_requirements[j]
        if gpa >= min_gpa and sat >= min_sat and rank <= max_rank:
            eligibility_matrix[i, j] = 1
        constraints.append(x[i, j] <= eligibility_matrix[i, j])  # Ensure eligibility 

problem = cp.Problem(objective, constraints)
problem.solve(verbose=False)

print("Total preference score:", objective.value)
# Print the results with a star (*) for students admitted to their top university preference
print("Matching results:")
for i, original_index in enumerate(sorted_indices):
    admitted_universities = []
    top_preference_universities = [j + 1 for j in range(num_universities) if sorted_preference_scores[i, j] == 3]  # 1-indexed
    
    for j in range(num_universities):
        if x.value is not None and x.value[i, j] > 0.5:
            admitted_universities.append(j + 1)
    
    if admitted_universities:
        stars = ["*" if uni in top_preference_universities else "" for uni in admitted_universities]
        results = ", ".join(f"{uni}{star}" for uni, star in zip(admitted_universities, stars))
        print(f"Student {original_index + 1} admitted to University/Universities: {results}")
    else:
        print(f"Student {original_index + 1} not admitted to any university")


Total preference score: 1057.0
Matching results:
Student 70 admitted to University/Universities: 1*, 2*, 3*, 4, 5*, 7, 8, 9, 10, 12*, 13, 14, 15*, 16*, 17*, 19, 20*
Student 51 admitted to University/Universities: 3*, 5*, 14*, 17, 19*, 20*
Student 12 admitted to University/Universities: 2, 3, 5, 8, 9*, 14, 17, 19, 20
Student 35 admitted to University/Universities: 2*, 3*, 5, 7*, 8, 9, 10, 13, 14, 16, 17, 19*, 20*
Student 2 admitted to University/Universities: 2, 3, 5*, 7, 8, 9*, 14, 17, 19*, 20
Student 34 admitted to University/Universities: 2, 3, 4, 5*, 7*, 8, 9, 10, 12*, 13*, 14, 16, 17*, 19*, 20
Student 53 admitted to University/Universities: 2, 3, 5, 7, 8, 9*, 14*, 17, 19, 20*
Student 56 admitted to University/Universities: 2, 3*, 4, 5*, 7, 8, 9*, 10, 12*, 13, 14*, 16*, 17, 19*, 20*
Student 44 admitted to University/Universities: 2, 3*, 4*, 5*, 7, 8*, 9, 10, 12*, 13*, 14, 15*, 16, 17, 19*, 20*
Student 54 admitted to University/Universities: 2, 3, 4, 7*, 8, 9*, 12*, 13, 14, 15*, 16, 